# 4. **Processing**
In this tutorial, we'll explain how to process data. Let's start\
with the data we acquired at the end of the last tutorial:

In [ ]:
using SeisIO, Dates
S = (
        if isfile("req_1.seis")
            rseis("req_1.seis")[1]
        else
            ds = Dates.now(); ds -= (Day(1) + Millisecond(ds) + Second(ds))
            s = string(ds)
            get_data("FDSN", "UW.MBW..EHZ, UW.SHW..EHZ, UW.HSR..EHZ, UW.TDH..EHZ, CC.PALM..EH?", src="IRIS", s=s, t=600) 
        end
    )
if isfile("req_1.seis") == false
    wseis("req_1.seis", S)
end

## A. **List of Data Processing Functions**
* convert_seis: differentiate and integrate seismograms
* demean: remove mean
* detrend: remove trend line
* env: compute envelope
* filtfilt: zero-phase filter
* nanfill: fill NaNs
* resample: ...what do you think it does?
* sync: time-synchronize a SeisData structure
* merge: merge data
* ungap: remove gaps in channels
* unscale: divide out :gain from all trace data

### **"In-Place" vs. "Safe" Operations**

Processing functions each have two versions: an \
"in-place" variant that ends with an exclamation mark ("!"),\
and a "safe" version that stores the result in a new data \
structure. The in-place variant overwrites data in place.\
This generally *isn't* reversible.\
\
Here's a simple set of processing operations, common for\
seismic data:

In [ ]:
U = deepcopy(S) # back up, just ... in... case...

In [ ]:
detrend!(S)
ungap!(S)
resample!(S, fs=50.0)
filtfilt!(S, rt="Lowpass", fh=10.0)
show_processing(S)

The filter uses a mix of default values and custom keyword parameters; \
the result applies a 4-pole Butterworth filter to each (regularly-sampled) \
channel of S in place.\
\
We've added a simple function to check for (and remove) problematic data.\
Let's load and test it. Note that this test *should* produce warnings.

In [ ]:
include("check_data.jl")

try
  test_check_tutorial_data()
  @info(string("Test of check_tutorial_data! passed."))
catch err
  @warn(string("Test of check_tutorial_data! threw error:", err))
end

Now let's check our test data set for errors.

In [ ]:
println("checking tutorial data now...")
check_tutorial_data!(S)

Assuming S is non-empty, we'll proceed.

In [ ]:
isempty(S)

We want at least four channels in S for the next section.\
If we don't have that, remember: data outages happen. We were\
requesting near-real-time data from the IRIS FDSN server. So,\
we can either (a) retry the data request at some later time, or\
(b) generate a fictitious data set. The next step uses option (b).

In [ ]:
if S.n < 4
    n = 4-S.n
    @warn("Too few channels for Instrument Response tutorial. Adding ", n, " channels of fictitious data.")
    S2 = randSeisData(n, nx=2^16, s=1.0, a0=true, fs_min=10.0)
    append!(S, S2)
else
    println("OK to proceed. No fictitious data appended.")
end
S

### **Instrument Response**
Translating instrument responses can be tricky. Let's work an\
example of that next. First, we create a target instrument response.\
Since the data in S are mostly short-period geophones, let's\
keep it reasonable -- a 10s rolloff frequency.

In [ ]:
?fctoresp

In [ ]:
resp_new = fctoresp(0.1)

\
Now let's update the sensitivity, so that the translated\
instrument responses have meaningful velocity amplitudes.

In [ ]:
resp_a0!(resp_new)
resp_new

In [ ]:
?translate_resp

\
Update the first three channels of S to the new response:

In [ ]:
translate_resp!(S, resp_new, chans=1:3)

\
Remove the instrument response from the rest completely, flattening to DC:

In [ ]:
Sf = remove_resp(S, chans=4:S.n)

\
How do the data look?

In [ ]:
check_tutorial_data!(S)

We need at least two channels left in S for the last section.\
If we don't have that, we should retry our data request at some\
later time.

In [ ]:
if S.n < 2
    @warn("Too few channels for Synchronization tutorial.")
    println("Please re-send the data request at a later time, or try again with different parameters.")
else
    println("OK to proceed.")
end

### **Synchronization**
In some situations, it's useful to force all traces in a `SeisData` \
container to start (and perhaps end) at the same time(s). Do this \
with the command `sync`:

In [ ]:
S = deepcopy(U)

In [ ]:
# don't worry about memorizing these commands; they're
# only for demonstration purposes.
d0 = u2d(1.0e-6*(minimum([S.t[i][1,2] for i in 1:S.n]) - 1000000))
d0 -= Millisecond(d0)
d0 += Millisecond(5)
t0 = string(d0)
t1 = string(d0+Second(300))

Synchronized traces can still have trace lengths \
that differ by a sample if the start and end times \
align exactly with one trace's sample intervals but \
not another's; that's why we added 5 ms to t0 above.

In [ ]:
sync!(S, s=t0, t=t1)

In [ ]:
[length(i) for i in S.x]

...are they all equal? They should be within two samples.

### **Check logs early and often**
Like file I/O, all data processing operations are logged to the `:notes` \
fields of SeisIO data structures. For example:

In [ ]:
?show_processing

In [ ]:
show_processing(S)

## B. **Cleanup**

In [ ]:
rm("req_1.seis")

## C. **For More Help**
Please consult the Processing chapter of the official documentation: \
https://seisio.readthedocs.io/en/latest/src/Processing/processing.html